<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3906341d9f47265352b1d452b1c15db57a554d411d9ebc98b495eb0e717741b0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 15:18:43
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: -98.74 K (-0.7%)
Current PnL: -22.16 L (-14.77%)
CY Booked + Current PnL: -10.26 L (-6.84%)
-------------------
Total profit:  1.17 L
Total loss:  -23.33 L
-------------------
Total Booked + Current PnL: 16.72 L (13.58%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.54%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.37 L (64.15%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.37%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.74,212871.0,8371.0,14922.0,-0.63,4.09,7.01,11.39,10.0,0.56,1.54,13.41,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.80,50.0,M-SC,1.30,84750.0,-16027.0,16119.0,1.50,-15.90,19.02,0.09,245.0,-0.99,0.61,11.34,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208605.0,16863.0,20902.0,0.89,8.79,10.02,19.69,99.0,0.81,1.51,16.19,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.85,214578.0,4212.0,24569.0,0.04,2.00,11.45,13.68,37.0,0.17,1.55,18.86,XY25,NTT,REFINERIES
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195233.0,-4905.0,25790.0,-0.88,-2.45,13.21,10.44,4.0,-0.19,1.41,2.98,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.00,-36.29,64.0,M-SC,12.85,79336.0,-21823.0,96711.0,4.04,-21.57,121.90,74.03,202.0,-0.23,0.57,11.87,AR,BTT,AUTO
4,ANGELONE,3033.00,11.73,64.0,X-SC,10.32,197382.0,6376.0,51319.0,2.84,3.34,26.00,30.21,157.0,0.12,1.43,24.92,X40N,NTT,FINANCE
45,KPIGREEN,731.05,-4.80,45.0,H-SC,6.23,113589.0,-11708.0,70641.0,1.84,-9.34,62.19,47.03,141.0,-0.17,0.82,42.22,MH,ATH,POWER
76,TTKPRESTIG,770.00,96.80,50.0,M-SC,1.30,84750.0,-16027.0,16119.0,1.50,-15.90,19.02,0.09,245.0,-0.99,0.61,11.34,OX40N,NTT,DURABLES
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.50,148257.0,-963.0,145144.0,1.43,-0.65,97.90,96.62,208.0,-0.01,1.07,60.45,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,-277.79,23.0,H-SC,3.11,98685.0,-64920.0,186238.0,-4.99,-39.68,188.72,74.15,139.0,-0.35,0.71,26.50,XR,NTT,CHEMICALS
23,DEN,75.0,86.53,35.0,M-SC,3.91,52560.0,-31904.0,67440.0,-4.56,-37.77,128.31,42.07,237.0,-0.47,0.38,11.62,AR,NTT,ENTERTAINMENT
25,GREENPANEL,537.0,210.72,45.0,M-SC,4.37,138616.0,-44462.0,123438.0,-3.78,-24.29,89.05,43.14,230.0,-0.36,1.00,30.39,XY24,NTT,MISC
10,BANDHANBNK,400.0,-12.63,47.0,H-SC,2.96,217857.0,-60702.0,318137.0,-3.50,-21.79,146.03,92.42,151.0,-0.19,1.58,25.90,XY24,NTT,BANKS
68,SURYODAY,240.0,57.89,54.0,H-SC,10.46,145472.0,-33599.0,99328.0,-3.23,-18.76,68.28,36.71,135.0,-0.34,1.05,44.34,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.5,148257.0,-963.0,145144.0,1.43,-0.65,97.90,96.62,208.0,-0.01,1.07,60.45,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.3,123676.0,340.0,131282.0,-2.71,0.28,106.15,106.72,119.0,0.00,0.89,23.72,AR,ATH,MISC
54,RECLTD,446.00,45.36,43.0,M-LC,6.2,203858.0,688.0,41444.0,-0.12,0.34,20.33,20.74,55.0,0.02,1.47,5.90,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,103.03,55.0,H-SC,11.51,128304.0,-14175.0,33693.0,-2.50,-9.95,26.26,13.70,163.0,-0.42,0.93,51.43,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.80,50.0,M-SC,1.30,84750.0,-16027.0,16119.0,1.50,-15.90,19.02,0.09,245.0,-0.99,0.61,11.34,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220716.0,-48951.0,85285.0,-0.19,-18.15,38.64,13.47,138.0,-0.57,1.60,12.23,XY24,NTT,PAINTS
17,CERA,9475.0,-23.63,41.0,H-SC,1.98,139414.0,-36489.0,78504.0,0.82,-20.74,56.31,23.89,149.0,-0.46,1.01,20.40,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.89,42.0,H-MC,7.02,101526.0,-30309.0,72530.0,-0.16,-22.99,71.44,32.03,98.0,-0.42,0.73,13.72,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203858.0,688.0,41444.0,-0.12,0.34,20.33,20.74,55.0,0.02,1.47,5.90,XY25,NTT,FINANCE
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219762.0,14686.0,97882.0,-0.07,7.16,44.54,54.89,52.0,0.15,1.59,32.98,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,58.0,M-LC,9.39,159066.0,18226.0,26262.0,0.45,12.94,16.51,31.59,66.0,0.69,1.15,42.86,XY24,NTT,BANKS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,153952.0,3007.0,106442.0,1.30,1.99,69.14,72.51,53.0,0.03,1.11,8.13,XR,NTT,IT
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191250.0,11442.0,100961.0,-1.98,6.36,52.79,62.52,88.0,0.11,1.38,42.62,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219762.0,14686.0,97882.0,-0.07,7.16,44.54,54.89,52.0,0.15,1.59,32.98,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191250.0,11442.0,100961.0,-1.98,6.36,52.79,62.52,88.0,0.11,1.38,42.62,XR,NTT,BANKS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,153952.0,3007.0,106442.0,1.30,1.99,69.14,72.51,53.0,0.03,1.11,8.13,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.30,123676.0,340.0,131282.0,-2.71,0.28,106.15,106.72,119.0,0.00,0.89,23.72,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.62,30.0,M-SC,3.40,86895.0,-4604.0,83358.0,-0.48,-5.03,95.93,86.07,223.0,-0.06,0.63,26.19,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175857.0,-24649.0,121799.0,-1.37,-12.29,69.26,48.45,90.0,-0.20,1.27,1.41,X40N,ATH,FINANCE
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226138.0,-21593.0,96651.0,-1.60,-8.72,42.74,30.30,92.0,-0.22,1.64,3.42,X40,ATH,ELECTRICAL
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51345.0,-13661.0,164592.0,-1.70,-21.01,320.56,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
5,ASIANPAINT,3465.66,-19.07,31.0,X-LC,5.31,204178.0,-47590.0,100803.0,-0.95,-18.90,49.37,21.13,27.0,-0.47,1.48,9.83,X40,ATH,PAINTS
55,RELAXO,1176.00,-41.14,34.0,X-SC,3.33,80276.0,-64884.0,140812.0,-1.15,-44.70,175.41,52.31,136.0,-0.46,0.58,7.45,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,43.0,H-LC,0.94,155350.0,-30745.0,78219.0,0.96,-16.52,50.35,25.51,15.0,-0.39,1.12,14.48,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.03,208605.0,16863.0,20902.0,0.89,8.79,10.02,19.69,99.0,0.81,1.51,16.19,XY25,NTT,AC
72,TATAMOTORS,1065.00,-24.14,37.0,X-LC,1.17,239852.0,-36671.0,146741.0,-2.68,-13.26,61.18,39.81,54.0,-0.25,1.74,14.95,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.73,29.0,X-MC,1.37,226138.0,-21593.0,96651.0,-1.60,-8.72,42.74,30.30,92.0,-0.22,1.64,3.42,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.34,43.0,X-MC,1.47,137048.0,915.0,30205.0,1.01,0.67,22.04,22.86,91.0,0.03,0.99,13.37,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51345.0,-13661.0,164592.0,-1.70,-21.01,320.56,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175857.0,-24649.0,121799.0,-1.37,-12.29,69.26,48.45,90.0,-0.20,1.27,1.41,X40N,ATH,FINANCE
82,VBL,671.64,-21.70,37.0,X-LC,10.51,281868.0,-33974.0,146628.0,-0.15,-10.76,52.02,35.67,5.0,-0.23,2.04,1.61,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.64,36.0,X-MC,7.68,220260.0,-43105.0,156451.0,-0.84,-16.37,71.03,43.04,84.0,-0.28,1.59,2.26,XY25,ATH,FMCG
79,UNITDSPR,1644.00,-14.51,42.0,X-LC,6.47,224082.0,-10062.0,58687.0,-1.01,-4.30,26.19,20.77,86.0,-0.17,1.62,2.32,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.85,51345.0,-13661.0,164592.0,-1.70,-21.01,320.56,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.14,34.0,X-SC,3.33,80276.0,-64884.0,140812.0,-1.15,-44.70,175.41,52.31,136.0,-0.46,0.58,7.45,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.44,34.0,X-SC,4.35,93466.0,-35204.0,80502.0,-1.99,-27.36,86.13,35.20,219.0,-0.44,0.68,7.28,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.94,43.0,X-SC,1.05,105846.0,3842.0,40084.0,-2.28,3.77,37.87,43.06,122.0,0.10,0.77,21.53,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.21,108570.0,-19368.0,66499.0,0.10,-15.14,61.25,36.84,143.0,-0.29,0.79,11.15,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-30.29,43.0,X-LC,12.30,278353.0,-67603.0,135836.0,-1.53,-19.54,48.80,19.73,1.0,-0.50,2.01,2.52,X40,ATH,IT
39,INFY,2275.00,-18.52,52.0,X-LC,7.50,318052.0,5164.0,166532.0,0.00,1.65,52.36,54.87,3.0,0.03,2.30,8.41,X40,BTT,IT
41,ITC,452.00,-39.61,41.0,X-LC,2.56,195233.0,-4905.0,25790.0,-0.88,-2.45,13.21,10.44,4.0,-0.19,1.41,2.98,X40,NTT,FMCG
82,VBL,671.64,-21.70,37.0,X-LC,10.51,281868.0,-33974.0,146628.0,-0.15,-10.76,52.02,35.67,5.0,-0.23,2.04,1.61,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,41.0,H-LC,7.29,245616.0,-16003.0,135212.0,-0.59,-6.12,55.05,45.57,7.0,-0.12,1.78,4.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,50.0,L-SC,28.74,81562.0,-31987.0,72036.0,-0.26,-28.17,88.32,35.27,268.0,-0.44,0.59,99.62,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.92,89700.0,-8280.0,29879.0,-0.94,-8.45,33.31,22.05,152.0,-0.28,0.65,30.39,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,103.03,55.0,H-SC,11.51,128304.0,-14175.0,33693.0,-2.50,-9.95,26.26,13.70,163.0,-0.42,0.93,51.43,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.46,145472.0,-33599.0,99328.0,-3.23,-18.76,68.28,36.71,135.0,-0.34,1.05,44.34,XR,NTT,BANKS
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.50,148257.0,-963.0,145144.0,1.43,-0.65,97.90,96.62,208.0,-0.01,1.07,60.45,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-185.23,67.0,M-SC,17.50,148257.0,-963.0,145144.0,1.43,-0.65,97.90,96.62,208.0,-0.01,1.07,60.45,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.46,145472.0,-33599.0,99328.0,-3.23,-18.76,68.28,36.71,135.0,-0.34,1.05,44.34,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219762.0,14686.0,97882.0,-0.07,7.16,44.54,54.89,52.0,0.15,1.59,32.98,X5K,ATH,METALS
50,PGHH,17907.65,-26.18,62.0,X-MC,2.92,213750.0,12930.0,54870.0,0.50,6.44,25.67,33.76,80.0,0.24,1.55,11.97,X40,ATH,FMCG
4,ANGELONE,3033.00,11.73,64.0,X-SC,10.32,197382.0,6376.0,51319.0,2.84,3.34,26.00,30.21,157.0,0.12,1.43,24.92,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.30
1,25,43.82
2,50,75.99


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.17
LC    34.11
MC    21.70
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.19
X40      14.74
XY25     12.01
XR       10.91
X40N     10.04
AR        8.16
OX40N     7.63
X200      1.78
X5K       1.59
SR        1.11
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.44
X-LC    23.09
X-MC    16.01
M-SC    12.67
M-LC     5.32
H-LC     4.68
X-SC     4.62
H-MC     3.77
M-MC     1.61
L-SC     1.44
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,-6.40,41.47
IT,12.94,-17.48,81.12
FINANCE,12.08,-13.05,64.29
MISC,6.86,-21.12,88.22
BANKS,6.40,-12.80,72.13
PAINTS,5.58,-19.76,37.61
ELECTRICAL,5.33,-11.92,51.19
INSURANCE,3.81,-5.03,44.29
AUTO,3.54,-17.60,73.81


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3262516.0,22
XR,1401622.0,14
AR,1200993.0,9
X40,803771.0,10
X40N,640750.0,8
OX40N,590744.0,10
XY25,521073.0,8
SR,267514.0,2
X5K,97882.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3483875.0,25
M-SC,1288496.0,15
X-MC,1113801.0,11
X-LC,1048943.0,13
X-SC,543808.0,6
H-MC,357701.0,3
H-LC,292660.0,3
M-LC,272030.0,4
L-SC,261822.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213810.0      6
           XR         829725.0      7
           AR         823411.0      5
M-SC       XY24       696901.0      6
X-MC       XY24       589487.0      4
X-LC       X40        498107.0      5
X-SC       X40N       298714.0      4
H-SC       OX40N      278774.0      4
           SR         267514.0      2
X-MC       X40        225162.0      4
X-LC       X40N       220237.0      3
           XY24       216714.0      2
H-LC       AR         213431.0      2
H-MC       XY24       184210.0      1
X-MC       XY25       177353.0      2
L-SC       XR         171656.0      2
M-MC       XY24       170540.0      1
X-SC       XY24       164592.0      1
M-SC       AR         164151.0      2
           OX40N      149274.0      4
           XY25       145144.0      1
           XR         133026.0      2
X-MC       X40N       121799.0      1
X-LC       XY25       113885.0      3
M-LC       XR         106442.0      1
H-MC       XR         100961.0      1
M-LC       X5K         97882.0      1
L-SC       OX40N       90166.0      1
X-SC       X40         80502.0      1
H-LC       X200        79229.0      1
H-MC       OX40N       72530.0      1
H-SC       MH          70641.0      1
L-MC       XR          59812.0      1
L-LC       XY25        43247.0      1
M-LC       XY25        41444.0      1
           XY24        26262.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
